**Generates plot of signature concordance**

In [2]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
import math

from collections import Counter
#pathPrefix = '/Users/friedman/Desktop/mnt'
#sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
#import analysis_utils 
#import mutationSigUtils

## FIGURE 3A

In [ ]:
#30 SIGS VERSION
stratonSpectra = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/noahFirstProject/signature_sig_copy/mutation-signatures/Stratton_signatures30.txt')
listOfDicts = []
oneBinFrac = 1.0/96 
stronglyFavoredFrac = .1
for signature in stratonSpectra.index:
    row = stratonSpectra[stratonSpectra.index == signature]
    nGreaterThanFrac = sum([1 if i > oneBinFrac else 0 for i in row.values[0]])
    fracStronglyFavored = sum([i if i > stronglyFavoredFrac else 0 for i in row.values[0]])
    listOfDicts.append({'Signature': signature, 'nbin/96': nGreaterThanFrac, 'fracStronglyFavored': fracStronglyFavored})
df = pd.DataFrame(listOfDicts) 
df.to_csv('/Users/friedman/Desktop/exomeProjectHelp/sigBinsInfo.tsv', index=False, sep='\t') 

In [ ]:
#60 SIGS VERSION
v3Spectra = pd.read_csv('/Users/friedman/Desktop/exomeProjectHelp/sigProfiler_exome_SBS_signatures.csv')
listOfDicts = []
oneBinFrac = 1.0/96 
stronglyFavoredFrac = .1
for signature in set(v3Spectra.columns.values) - set(['Type', 'SubType']):
    col = list(v3Spectra[signature])
    nGreaterThanFrac = sum([1 if i > oneBinFrac else 0 for i in col])
    fracStronglyFavored = sum([i if i > stronglyFavoredFrac else 0 for i in col])
    listOfDicts.append({'Signature': signature, 'nbin/96': nGreaterThanFrac, 'fracStronglyFavored': fracStronglyFavored})
df = pd.DataFrame(listOfDicts) 
df.to_csv('/Users/friedman/Desktop/exomeProjectHelp/sigBinsInfo60Sigs.tsv', index=False, sep='\t') 


## FIGURE 3B

In [12]:
#These functions format the data as needed for plotting

#EXPANDS THE COLUMNS AS NEEDED
def expand_df(df, mode):
    listOfDicts = []
    for index, row in df.iterrows():
        #localD = {
        #    'Tumor_Sample_Barcode': row['Tumor_Sample_Barcode'], 'CMOID':row['CMOID'], 'TMBExome': row['TMBExome'], 'TMBIMPACT': row['TMBIMPACT'], 'MSIIMPACT': row['MSIIMPACT'], 'MSIExome': row['MSIExome'],
        #    'SigIMPACTPresent':row['SigIMPACTPresent'], 'SigWESPresent':row['SigWESPresent'], 'Cancer_Type':row['Cancer_Type']
        #}
        localD = {
            'Tumor_Sample_Barcode': row['Tumor_Sample_Barcode'], 'DMP':row['DMP'],
            'TMB': row['TMB'], 'Cancer_Type':row['Cancer_Type_Aggregate'],
            'type': row['type']
        }
        
        if not isinstance(row['impactSig'], float):
            impactSigsSplit = row['impactSig'].split(',')

            #['Insufficient_Mutations']
            if len(impactSigsSplit) > 1:
                for sig in impactSigsSplit:
                    sigName = sig.split(':')[0].strip(' ')
                    fraction = float(sig.split(':')[1].split('|')[0]) #SAVE THE FRACTION
                    confidence = float(sig.split(':')[1].split('|')[1]) #SAVE THE CONFIDENCE
                    localD['impactSig_' + sigName] = fraction
                    localD['impactConfidence_' + sigName] = confidence
                    localD['insufficientImpactMutBurden'] = False
        else:
            localD['insufficientImpactMutBurden'] = True
        
        if not isinstance(row['exomeSig'], float):
            exomeSigsSplit = row['exomeSig'].split(',')
            if len(exomeSigsSplit) > 1:
                for sig in exomeSigsSplit:
                    sigName = sig.split(':')[0].strip(' ')

                    frac, conf = sig.split(':')[1].split('|')
                    #TODO test for NA

                    if frac == ' NA': frac = 0  
                    if conf == ' NA': conf = 0

                    fraction = float(frac)
                    confidence = float(conf) #SAVE THE CONFIDENCE
                    localD['exomeSig_' + sigName] = fraction
                    localD['exomeConfidence_' + sigName] = confidence
                localD['insufficientExomeMutBurden'] = False  
            else:
                localD['insufficientExomeMutBurden'] = True
        else:
            localD['insufficientExomeMutBurden'] = True
        listOfDicts.append(localD)
    df = pd.DataFrame(listOfDicts)
    
    #MERGE SIGNATURE COLUMNS AS I LIKE:
    if mode == '30sigs':
        df['impactSig_AID/APOBEC'] = df['impactSig_AID/APOBEC_2'] + df['impactSig_AID/APOBEC_13']
        df['exomeSig_AID/APOBEC'] = df['exomeSig_AID/APOBEC_2'] + df['exomeSig_AID/APOBEC_13']

        df['impactSig_MMR/MSI'] = df['impactSig_MMR/MSI_20'] + df['impactSig_MMR/MSI_15'] + df['impactSig_MMR/MSI_21'] + df['impactSig_MMR/MSI_26'] + df['impactSig_MMR/MSI_6']
        df['exomeSig_MMR/MSI'] = df['exomeSig_MMR/MSI_20'] + df['exomeSig_MMR/MSI_15'] + df['exomeSig_MMR/MSI_21'] + df['exomeSig_MMR/MSI_26'] + df['exomeSig_MMR/MSI_6']

        df = df.drop(['impactSig_AID/APOBEC_2', 'impactSig_AID/APOBEC_13',
                      'exomeSig_AID/APOBEC_2', 'exomeSig_AID/APOBEC_13',
                      'impactSig_MMR/MSI_20', 'impactSig_MMR/MSI_15', 'impactSig_MMR/MSI_21', 'impactSig_MMR/MSI_26', 'impactSig_MMR/MSI_6',
                      'exomeSig_MMR/MSI_20', 'exomeSig_MMR/MSI_15', 'exomeSig_MMR/MSI_21', 'exomeSig_MMR/MSI_26', 'exomeSig_MMR/MSI_6'],
                      axis=1)
    #adjust it according to 60 sig rule    
    else:
        #rename some signatures
        df['impactSig_BRCA1/2'] = df['impactSig_SBS3'] 
        df['exomeSig_BRCA1/2'] = df['exomeSig_SBS3'] 
        
        df['impactSig_Aging'] = df['impactSig_SBS1'] 
        df['exomeSig_Aging'] = df['exomeSig_SBS1'] 
        
        df['impactConfidence_BRCA1/2'] = df['impactConfidence_SBS3']
        df['exomeConfidence_BRCA1/2'] = df['exomeConfidence_SBS3']
        
        #combine signatures
        df['impactSig_AID/APOBEC'] = df['impactSig_SBS2'] + df['impactSig_SBS13']
        df['exomeSig_AID/APOBEC'] = df['exomeSig_SBS2'] + df['exomeSig_SBS13']
        
        df['impactSig_SMOKING'] = df['impactSig_SBS4'] + df['impactSig_SBS24'] + df['impactSig_SBS29']
        df['exomeSig_SMOKING'] = df['exomeSig_SBS4'] + df['exomeSig_SBS24'] + df['exomeSig_SBS29']
        
        df['impactSig_MMR/MSI'] = df['impactSig_SBS6'] + df['impactSig_SBS15'] + df['impactSig_SBS21'] + df['impactSig_SBS26']
        df['exomeSig_MMR/MSI'] = df['exomeSig_SBS6'] + df['exomeSig_SBS15'] + df['exomeSig_SBS21'] + df['exomeSig_SBS26']
        
        df['impactSig_UV'] = df['impactSig_SBS7a'] + df['impactSig_SBS7b'] + df['impactSig_SBS7c'] + df['impactSig_SBS7d']
        df['exomeSig_UV'] = df['exomeSig_SBS7a'] + df['exomeSig_SBS7b'] + df['exomeSig_SBS7c'] + df['exomeSig_SBS7d']
        
        df['impactSig_POLE'] = df['impactSig_SBS10a'] + df['impactSig_SBS10b']
        df['exomeSig_POLE'] = df['exomeSig_SBS10a'] + df['exomeSig_SBS10b']
        
        df = df.drop(['impactSig_SBS1', 'exomeSig_SBS1',
                      'impactSig_SBS3', 'exomeSig_SBS3',
                      'impactSig_SBS4', 'impactSig_SBS24', 'impactSig_SBS29', 'exomeSig_SBS4', 'exomeSig_SBS24', 'exomeSig_SBS29',
                      'impactSig_SBS2', 'impactSig_SBS13','exomeSig_SBS2', 'exomeSig_SBS13',
                      'impactSig_SBS6', 'impactSig_SBS15', 'impactSig_SBS21', 'impactSig_SBS26','exomeSig_SBS6', 'exomeSig_SBS15', 'exomeSig_SBS21', 'exomeSig_SBS26',
                      'impactSig_SBS7a', 'impactSig_SBS7b', 'impactSig_SBS7c', 'impactSig_SBS7d','exomeSig_SBS7a', 'exomeSig_SBS7b', 'exomeSig_SBS7c', 'exomeSig_SBS7d',
                      'impactSig_SBS10a', 'impactSig_SBS10b', 'exomeSig_SBS10a', 'exomeSig_SBS10b',
                     'impactConfidence_SBS3', 'exomeConfidence_SBS3'], 
            axis=1)
        
    return df



def set_up_data(mode='30sigs'):
    shwetaSigDataExome = pd.read_table('/Users/friedman/Desktop/TempoSampleLevel_WES.tsv')
    shwetaSigDataImpact = pd.read_table('/Users/friedman/Desktop/TempoSampleLevel_IMPACT.tsv')

    if mode == '30sigs':
        shwetaSigDataExome['exomeSig'] = shwetaSigDataExome['Signatures']
        shwetaSigDataImpact['impactSig'] = shwetaSigDataImpact['Signatures']
    else:
        shwetaSigDataExome['exomeSig'] = shwetaSigDataExome['SignaturesSBS60R2']
        shwetaSigDataImpact['impactSig'] = shwetaSigDataImpact['SignaturesSBS60R2']

    shwetaSigDataExome['type'] = 'Exome'
    shwetaSigDataImpact['type'] = 'Impact'
    shwetaSigData = pd.concat([shwetaSigDataExome, shwetaSigDataImpact])
    
    #expand the data frame to the format I want and mark dominant impact and exome signatures
    expandedDf = expand_df(shwetaSigData, mode)
    
    expandedDf['DominantSignatureImpact'] = expandedDf.apply(lambda row: get_dominant_signature(row.to_dict(), prefix='impactSig'), axis=1)
    expandedDf['DominantSignatureExome'] = expandedDf.apply(lambda row: get_dominant_signature(row.to_dict(), prefix='exomeSig'), axis=1)
    return expandedDf


#gets the dominant signature in a case
def get_dominant_signature(rowAsDict, prefix='impactSig'):
    cols = [x for x in rowAsDict.keys() if prefix in x]
    tupList = []
    for key, value in rowAsDict.items():
        if prefix in key: tupList.append((key, value))
    sortedSigs = sorted(tupList, key = lambda tup: tup[1], reverse=True)
    
    if math.isnan(sortedSigs[0][1]):
        return 'notEnoughMutations'
    else:
        return sortedSigs[0][0]

#returnMode spex
#slightly a misnomer finds the nth biggest non signature of interest and returns it
#n is 1 indexed for readability so we subtract 1
def find_nth_biggest_non_primary_sig(row, primarySig, n, prefix='Signature.'):
    colNames = row.to_dict().keys()

    signatureColumns = [i for i in colNames if prefix in i]
    rowSigsOnly = row[signatureColumns]
    rowAsDict = rowSigsOnly.to_dict()
    items = rowAsDict.items()
    sortedItems = sorted(items, key=lambda x: x[1], reverse=True)

    #remove the dominant signature
    sortedItemsLimited = [z for z in sortedItems if z[0] != primarySig]
    return sortedItemsLimited[n - 1][0], sortedItemsLimited[n - 1][1]

def summarize_brca_sig_info(expandedDf, brcaBiallelicBRCARelated):
    exomeDf = expandedDf[expandedDf['type'] == 'Exome']
    impactDf = expandedDf[expandedDf['type'] == 'Impact']
    
    domSigImpactDict = dict(zip(impactDf['DMP'], impactDf['DominantSignatureImpact']))
    impactBrcaConfidenceDict = dict(zip(impactDf['DMP'], impactDf['impactConfidence_BRCA1/2']))
    impactBrcaSigDict = dict(zip(impactDf['DMP'], impactDf['impactSig_BRCA1/2']))
    
    domSigExomeDict = dict(zip(exomeDf['DMP'], exomeDf['DominantSignatureExome']))
    exomeBrcaConfidenceDict = dict(zip(exomeDf['DMP'], exomeDf['exomeConfidence_BRCA1/2']))
    exomeBrcaSigDict = dict(zip(exomeDf['DMP'], exomeDf['exomeSig_BRCA1/2']))
    tmbDict = dict(zip(exomeDf['DMP'], exomeDf['TMB']))
    
    listOfDicts = []
    #these are the methods we use to mark cases
    
    #print impactBrcaConfidenceDict
    
    methods = ['dominantImpactSig', 'confidentInImpact', 'presentInImpact']
    for tsb in brcaBiallelicBRCARelated:
        if tsb in domSigImpactDict and tsb in impactBrcaConfidenceDict and tsb in impactBrcaSigDict:
            for method in methods:
                detected = 0
                if method == 'dominantImpactSig':
                    if domSigImpactDict[tsb] == 'impactSig_BRCA1/2':
                        detected = 1
                elif method == 'confidentInImpact':
                    if impactBrcaConfidenceDict[tsb] > .9:
                        detected = 1
                elif method == 'presentInImpact':
                    if impactBrcaSigDict[tsb] > 0.2:
                        detected = 1

                listOfDicts.append({'Tumor_Sample_Barcode': tsb, 'method': method,
                                       'detected': detected, 'TMB': tmbDict[tsb],
                                       'impactSig_BRCA1/2': impactBrcaSigDict[tsb], 'impactConfidence_BRCA1/2': impactBrcaConfidenceDict[tsb],
                                       'DominantSignatureImpact': domSigImpactDict[tsb],
                                        'exomeConfidence_BRCA1.2': exomeBrcaConfidenceDict[tsb]})

            
    return pd.DataFrame(listOfDicts)


In [14]:
#WE run the following analyses all in two separate modes: 30sigs and 60sigs
#60sigs takes a while

df30Sigs = set_up_data(mode='30sigs')
#df60Sigs = set_up_data(mode='60sigs')

#df30Sigs['impactConfidence_']

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:112: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:113: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:124: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



KeyError: 'impactSig_AID/APOBEC_2'

In [5]:
philipData = pd.read_table('/Users/friedman/Desktop/exomeProjectHelp/gray-june-exome-table.txt')
brcaBiallelicBRCARelated = set(philipData[
(philipData['biallelic_class'].isin(set(['Germline biallelic', 'Somatic biallelic'])))
&(philipData['hrd_cancer'] == 'HRD')]['dmp_sample'])


#brcaPlottingDf30 = summarize_brca_sig_info(df30Sigs, brcaBiallelicBRCARelated)
#brcaPlottingDf60 = summarize_brca_sig_info(df60Sigs, brcaBiallelicBRCARelated)

#df60Sigs

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [6]:
len(brcaBiallelicBRCARelated)
#THIS NUMBER IS USED FOR PLOTTING

133

In [45]:

#brcaPlottingDf30 = brcaPlottingDf30.fillna(0)
#brcaPlottingDf30.to_csv('/Users/friedman/Desktop/exomeProjectHelp/data_3b_30sigs.tsv', index=False, sep='\t')
brcaPlottingDf60 = brcaPlottingDf60.fillna(0)
brcaPlottingDf60.to_csv('/Users/friedman/Desktop/exomeProjectHelp/data_3b_60sigs.tsv', index=False, sep='\t')


## FIGURE 3C

## FIGURE 3D

In [76]:
df = df60Sigs
#df = df30Sigs
impactDf = df[df['type'] == 'Impact']
exomeDf = df[df['type'] == 'Exome']
#Adjust the dominant signatures so that aflatoxin and signature 29 are marked as smoking
impactDf['DominantSignatureImpact'] = impactDf['DominantSignatureImpact'].apply(lambda x: 
            'impactSig_Smoking' if x == 'impactSig_Tobacco_chewing' or x == 'impactSig_Aflatoxin' else x)
exomeDf['DominantSignatureExome'] = exomeDf['DominantSignatureExome'].apply(lambda x: 
            'exomeSig_Smoking' if x == 'exomeSig_Tobacco_chewing' or x == 'exomeSig_Aflatoxin' else x)

domImpactSigDict = dict(zip(impactDf['DMP'], impactDf['DominantSignatureImpact']))
domExomeSigDict = dict(zip(exomeDf['DMP'], exomeDf['DominantSignatureExome']))



/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
df['dominantSignaturesAgree'] = df.apply(lambda row: 
    1 if domImpactSigDict[row['DMP']][10:] == domExomeSigDict[row['DMP']][9:]
                                                         else 0, axis=1) 
df['highMutationBurden'] = df['TMB'].apply(lambda x: True if x >= 20 else False)
#add a column for plotting dominant signature by marking many sigs as 'other'
df['dominantSignature'] = df['DominantSignatureExome'].apply(lambda x:
    x[9:] if x in set(['exomeSig_Aging', 'exomeSig_AID/APOBEC', 'exomeSig_BRCA1/2',
                                    'exomeSig_SMOKING', 'exomeSig_MMR/MSI', 'exomeSig_UV'])
                        else 'Other')

In [80]:
df.to_csv('/Users/friedman/Desktop/exomeProjectHelp/data_3d_signatureConcordance_60Sigs.tsv', index=False, sep='\t')

#print df60Sigs[df60Sigs['exomeSig_AID/APOBEC'] > .3][['exomeSig_AID/APOBEC', 'TMB', 'DominantSignatureExome', 'exomeSig_MMR/MSI']]

#print Counter(df['DominantSignatureExome'])
#print set(df['dominantSignature'])

#df[df['TMB'].isnull()]['DominantSignatureExome']

#df60Sigs.columns.values
#df60Sigs[['exomeSig_AID/APOBEC', 'TMB']]

#max(df60Sigs[df60Sigs['TMB'].notnull() & (df60Sigs['impactSig_AID/APOBEC'].notnull())]['impactSig_AID/APOBEC'])
#print df60Sigs[df60Sigs['impactSig_AID/APOBEC'].notnull()]['impactSig_AID/APOBEC']

#df[(df['impactSig_AID/APOBEC'] > .1) & (df['TMB'].notnull())][['dominantSignature', 'DominantSignatureExome', 'TMB']]

## FIGURE 3E

## Additional figure: why sbs doesn't work

In [133]:
df30Sigs = df30Sigs.fillna(0)
df60Sigs = df60Sigs.fillna(0)
df30SigsImpact = df30Sigs[df30Sigs['type'] == 'Impact']
df30SigsExome = df30Sigs[df30Sigs['type'] == 'Exome']
df60SigsImpact = df60Sigs[df60Sigs['type'] == 'Impact']
df60SigsExome = df60Sigs[df60Sigs['type'] == 'Exome']

sig3DictImpact = dict(zip(df30SigsImpact['DMP'], df30SigsImpact['impactSig_BRCA1/2']))
sbs3DictImpact = dict(zip(df60SigsImpact['DMP'], df60SigsImpact['impactSig_BRCA1/2']))
sbs39DictImpact = dict(zip(df60SigsImpact['DMP'], df60SigsImpact['impactSig_SBS39']))

sig3DictExome = dict(zip(df30SigsExome['DMP'], df30SigsExome['exomeSig_BRCA1/2']))
sbs3DictExome = dict(zip(df60SigsExome['DMP'], df60SigsExome['exomeSig_BRCA1/2']))
sbs39DictExome = dict(zip(df60SigsExome['DMP'], df60SigsExome['exomeSig_SBS39']))

listOfDicts = []
for case in brcaBiallelicBRCARelated & set(df30Sigs['DMP']) & set(df60Sigs['DMP']):
    listOfDicts.append({'type': 'IMPACT', 'val': sig3DictImpact[case], 'measure': '30sigs_sig3'})
    listOfDicts.append({'type': 'IMPACT', 'val': sbs3DictImpact[case], 'measure': '60sigs_SBS3'})
    listOfDicts.append({'type': 'IMPACT', 'val': sbs39DictImpact[case], 'measure': '60sigs_SBS39'})
    
    listOfDicts.append({'type': 'EXOME', 'val': sig3DictExome[case], 'measure': '30sigs_sig3'})
    listOfDicts.append({'type': 'EXOME', 'val': sbs3DictExome[case], 'measure': '60sigs_SBS3'})
    listOfDicts.append({'type': 'EXOME', 'val': sbs39DictExome[case], 'measure': '60sigs_SBS39'})
    
df = pd.DataFrame(listOfDicts)

In [134]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/signatureData.tsv', index=False, sep='\t')

In [132]:
#[x for y,x in dict(zip(df30Sigs['DMP'], df30Sigs['exomeSig_BRCA1/2'])) if x > 0]
#max(df30Sigs['exomeSig_BRCA1/2'])



0        Exome
1        Exome
2        Exome
3        Exome
4        Exome
5        Exome
6        Exome
7        Exome
8        Exome
9        Exome
10       Exome
11       Exome
12       Exome
13       Exome
14       Exome
15       Exome
16       Exome
17       Exome
18       Exome
19       Exome
20       Exome
21       Exome
22       Exome
23       Exome
24       Exome
25       Exome
26       Exome
27       Exome
28       Exome
29       Exome
         ...  
3242    Impact
3243    Impact
3244    Impact
3245    Impact
3246    Impact
3247    Impact
3248    Impact
3249    Impact
3250    Impact
3251    Impact
3252    Impact
3253    Impact
3254    Impact
3255    Impact
3256    Impact
3257    Impact
3258    Impact
3259    Impact
3260    Impact
3261    Impact
3262    Impact
3263    Impact
3264    Impact
3265    Impact
3266    Impact
3267    Impact
3268    Impact
3269    Impact
3270    Impact
3271    Impact
Name: type, Length: 3272, dtype: object